<a href="https://colab.research.google.com/github/Annie-Yeeun-Jang/Yeeun-J/blob/master/text_conf/make_data_by_discriminator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers

     |████████████████████████████████| 2.5MB 4.1MB/s 
     |████████████████████████████████| 3.3MB 24.8MB/s 
     |████████████████████████████████| 901kB 35.9MB/s 


# Train

In [4]:
import pandas as pd
import torch
import os
import matplotlib.pyplot as plt

# Preliminaries

from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
#source_folder = '/content/drive/My Drive/transformers/Data'


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
destination_folder = '/content/drive/Shareddrives/text_conf/discriminator/check_point/'

In [8]:
MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)
text_field = Field(use_vocab=False, tokenize=tokenizer.encode,is_target = True, lower=False, include_lengths=False, batch_first=True,
                   fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)


In [9]:
class BERT(nn.Module):

    def __init__(self):
        super(BERT, self).__init__()

        options_name = "bert-base-uncased"
        self.encoder = BertForSequenceClassification.from_pretrained(options_name)

    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]

        return loss, text_fea

In [10]:
model = BERT().to(device)
state_dict = torch.load(destination_folder + 'model.pt', map_location=device)
model.load_state_dict(state_dict['model_state_dict'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [11]:
tabular = TabularDataset(path="/content/drive/Shareddrives/text_conf/dataset/preprocessed/discriminator/with_academic_len32.csv", format='CSV', fields=[('text', text_field)], skip_header=True)
# Iterators
data_iter = Iterator(tabular, batch_size=16, device=device, train=False, shuffle=False, sort=False)

In [12]:
tokenizer.decode(next(iter(data_iter)).text[0])

'[CLS] the first thing i like about my best friend is that i can express my opinions and feelings to him comfortably. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [13]:
original = pd.read_csv("/content/drive/Shareddrives/text_conf/dataset/preprocessed/discriminator/with_academic_len32.csv").text.tolist()
original_batch = []
for batch in range(len(original)//16 + 1):
  original_batch.append(original[batch:(1+batch)*16])

In [14]:
threshold = 0.9
softmax = nn.Softmax(dim = 1)

In [15]:
with torch.no_grad():
  model.eval()
  native = []
  nonnative = []
  for (i, (iter)), original_text in zip(enumerate(data_iter), original_batch):
    text = iter.text.type(torch.LongTensor).to(device)
    label = torch.tensor([0]*len(text), dtype = torch.long).to(device)
    _, tmp = model(text, label)

    result = tmp.to('cpu')

    softmax_dist = softmax(result)
    argmax_idx = torch.argmax(softmax_dist, dim = 1)
    confidence_list = [dist[idx] for dist, idx in zip(softmax_dist, argmax_idx)]
    
    native_text = [line for line, idx, confidence in zip(original_text, argmax_idx, confidence_list) if idx == 0 and confidence > threshold]
    nonnative_text = [line for line, idx, confidence in zip(original_text, argmax_idx, confidence_list) if idx == 1 and confidence > threshold]

    native.extend(native_text)
    nonnative.extend(nonnative_text)

    if i % 300 == 0:
      print(f"{i}/{len(data_iter)} 배치 완료")

0/5680 배치 완료
300/5680 배치 완료
600/5680 배치 완료
900/5680 배치 완료
1200/5680 배치 완료
1500/5680 배치 완료
1800/5680 배치 완료
2100/5680 배치 완료
2400/5680 배치 완료
2700/5680 배치 완료
3000/5680 배치 완료
3300/5680 배치 완료
3600/5680 배치 완료
3900/5680 배치 완료
4200/5680 배치 완료
4500/5680 배치 완료
4800/5680 배치 완료
5100/5680 배치 완료
5400/5680 배치 완료


In [16]:
print(len(native), len(nonnative))

30363 52402


In [17]:
nonnative[-10:]

['Future research into faculty participation in athletics may suggest appropriate roles for greater faculty involvement in the overall institution environment .',
 'PREFORMATTED TABLE  b Significance found between Division III and both Division I and II at ALPHA .',
 'No significance found between Division I and II at ALPHA .',
 '@ NUM  Controversy exists concerning the efficacy and safety of creatine supplementation .',
 'The present study used a cross-sectional design to examine social cognitive predictors of creatine use versus non-use among  NUM  male , undergraduate , recreational resistance trainers .',
 'Seventy participants  reported creatine use .',
 'Creatine users had higher self-efficacy for resistance-training with creatine , t  =  NUM .',
 'NUM  , higher positive outcome expectancies for improved workout performance , t  =  NUM .',
 'NUM   NUM  , p = .',
 'NUM  , and lower negative outcome expectancies for creatine use , t  =  NUM .']

# train test val 나눠서 저장하기

In [18]:
native_idx = round(len(native)*0.9)
nonnative_idx = round(len(nonnative)*0.9)

In [19]:
import random
random.shuffle(native)
random.shuffle(nonnative)

native_train = native[:native_idx]
native_test = native[native_idx:]

nonnative_train = nonnative[:nonnative_idx]
nonnative_test = nonnative[nonnative_idx:]

In [20]:
native_train_pd = pd.DataFrame(native_train)
native_test_pd = pd.DataFrame(native_test)

nonnative_train_pd = pd.DataFrame(nonnative_train)
nonnative_test_pd = pd.DataFrame(nonnative_test)

native_train_pd.to_csv("/content/drive/Shareddrives/text_conf/dataset/styletransformer/native_train2.tsv", header=None, index = None)
native_test_pd.to_csv("/content/drive/Shareddrives/text_conf/dataset/styletransformer/native_test2.tsv", header=None, index = None)

nonnative_train_pd.to_csv("/content/drive/Shareddrives/text_conf/dataset/styletransformer/nonnative_train2.tsv", header=None, index = None)
nonnative_test_pd.to_csv("/content/drive/Shareddrives/text_conf/dataset/styletransformer/nonnative_test2.tsv", header=None, index = None)